## <center>Importações

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
from dotenv import load_dotenv
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer, util

### <center>Carregamento e transformação da Base

In [3]:
df=pd.read_excel("NPS-Comentarios-2024-Louveira_VilaAndrade.xlsx")

df = df.replace('-', pd.NA).dropna(how='all')
df = df.dropna()
df = df.drop(columns=["classificacao", "NPS"])

display(df)

Ano                     Centro  Mes  Semana Data Avaliação  \
0     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-01   
3     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
4     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
6     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
8     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-03   
...    ...                        ...  ...     ...            ...   
1075  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1076  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1078  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1079  2024             L5235-LOUVEIRA  dez      52     2024-12-27   
1082  2024             L5235-LOUVEIRA  dez       1     2024-12-30   

                                             Comentario  
0     Maior variedade, não tinha tudo o que eu procu...  
3                                       Nada a declarar  
4                 Tudo certinho por enquanto tah ótimo.  
6     Eu procurei paleta de cabrito mss a Swift nao ...  
8             Sempre sou muito bem atendida nesta loja.  
...                                                 ...  
1075  A loja de Louveira tem um dos melhores atendim...  
1076  O atendimento da moça que trabalha lá foi muit...  
1078                                       Mais ofertas  
1079  Mantendo a mesma  atenção com os Clientes!!!\n...  
1082                 Tudo perfeito  uma ótima  gerente.  

[570 rows x 6 columns]

## <center>Criando NPS

### <center>Criação do PipeLine de NPS

In [4]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    framework="pt"
)
 
def classificar_sentimento(texto):
    texto = texto.strip()
    if not texto or texto.lower() in ["nada a declarar"]:
        return "Neutro"
 
    result = sentiment_pipeline(texto)[0]
    estrelas = int(result['label'][0])
    
    if estrelas >= 4:
        return "Positivo"
    elif estrelas == 3:
        return "Neutro"
    else:
        return "Detrator"

C:\Users\marcusthomazetti-ieg\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marcusthomazetti-ieg\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 

### <center>Aplicando PipeLine NPS na Base

In [5]:
df["NPS"] = df["Comentario"].apply(classificar_sentimento)

display(df)

Ano                     Centro  Mes  Semana Data Avaliação  \
0     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-01   
3     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
4     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
6     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
8     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-03   
...    ...                        ...  ...     ...            ...   
1075  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1076  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1078  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1079  2024             L5235-LOUVEIRA  dez      52     2024-12-27   
1082  2024             L5235-LOUVEIRA  dez       1     2024-12-30   

                                             Comentario       NPS  
0     Maior variedade, não tinha tudo o que eu procu...  Detrator  
3                                       Nada a declarar    Neutro  
4                 Tudo certinho por enquanto tah ótimo.  Positivo  
6     Eu procurei paleta de cabrito mss a Swift nao ...  Detrator  
8             Sempre sou muito bem atendida nesta loja.  Positivo  
...                                                 ...       ...  
1075  A loja de Louveira tem um dos melhores atendim...  Positivo  
1076  O atendimento da moça que trabalha lá foi muit...  Positivo  
1078                                       Mais ofertas    Neutro  
1079  Mantendo a mesma  atenção com os Clientes!!!\n...  Detrator  
1082                 Tudo perfeito  uma ótima  gerente.  Positivo  

[570 rows x 7 columns]

## <center>Classificando comentários

### <center>Criando exemplos de classes

In [6]:
CATEGORY_EXAMPLES = {
    "Preço": [
        "Preço alto", "Mais promoções", "Descontos",
        "Preço muito caro", "Pagamento com vale refeição"
    ],
    "Estoque": [
        "Produto esgotado", "Falta estoque", "Acabou o produto",
        "Não encontrei o item"
    ],
    "Atendimento": [
        "Ótimo atendimento", "Funcionários atenciosos", "Equipe prestativa",
        "Agilidade e eficiência no atendimento"
    ],
    "Acesso": [
        "Difícil acesso", "Fila grande", "Estacionamento ruim"
    ],
    "Variedade dos Produtos": [
        "Faltam opções", "Pouca variedade", "Queria mais cortes",
        "Não tem o produto que procuro"
    ],
    "Limpeza": [
        "Loja suja", "Falta de higiene", "Ambiente limpo"
    ],
    "Localização": [
        "Local distante", "Localização ruim", "Fácil de chegar"
    ],
    "Segurança": [
        "Sensação de insegurança", "Lugar perigoso", "Pouca segurança"
    ],
    "Site": [
        "Site com erro", "App travando", "Problema na compra online"
    ],
    "Entrega": [
        "Demorou para entregar", "Atraso no delivery"
    ],
    "Nada a Declarar": [
        "Está ótimo", "Sem reclamações", "Não precisa melhorar",
        "Tudo perfeito"
    ],
    "Outros": [
        "Comentário genérico", "Sem categoria específica"
    ]
}

### <center>Criando PipeLine das Classes

In [7]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)


def build_centroids():
    centroids = {}
    for cat, examples in CATEGORY_EXAMPLES.items():
        emb = model.encode(examples, convert_to_tensor=True)
        centroids[cat] = emb.mean(dim=0)
    return centroids

CAT_CENTROIDS = build_centroids()


def classify_comment(text: str, threshold=0.40):
    emb = model.encode(text, convert_to_tensor=True)

    best_cat = None
    best_score = -1.0

    for cat, centroid in CAT_CENTROIDS.items():
        sim = float(util.cos_sim(emb, centroid)[0][0])
        if sim > best_score:
            best_score = sim
            best_cat = cat

    if best_score < threshold:
        return "Outros", best_score


    if best_cat == "Nada a Declarar":
        texto = text.lower()
        palavras_negativas = ["não", "ruim", "pior", "melhorar", "péssimo"]
        if any(p in texto for p in palavras_negativas):
            return "Outros", best_score

    return best_cat, best_score

C:\Users\marcusthomazetti-ieg\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marcusthomazetti-ieg\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' p

### <center>Aplicando PipeLine das classes na Base

In [8]:
def processar_classe():
    categorias = []
    scores = []

    for texto in df["Comentario"].astype(str):
        cat, score = classify_comment(texto)
        categorias.append(cat)

    df["Categoria"] = categorias

if __name__ == "__main__":
    processar_classe()

## <center>Gerando DF Final

In [9]:
df.to_excel("Comentarios_NPS_Classes.xlsx", index=False)
print(f"\nArquivo gerado: Comentarios_NPS_Classes.xlsx\n")
display(df)


Arquivo gerado: Comentarios_NPS_Classes.xlsx



Ano                     Centro  Mes  Semana Data Avaliação  \
0     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-01   
3     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
4     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
6     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-02   
8     2024  L5082-VILA ANDRADE (1346)  jan       1     2024-01-03   
...    ...                        ...  ...     ...            ...   
1075  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1076  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1078  2024             L5235-LOUVEIRA  dez      52     2024-12-26   
1079  2024             L5235-LOUVEIRA  dez      52     2024-12-27   
1082  2024             L5235-LOUVEIRA  dez       1     2024-12-30   

                                             Comentario       NPS  \
0     Maior variedade, não tinha tudo o que eu procu...  Detrator   
3                                       Nada a declarar    Neutro   
4                 Tudo certinho por enquanto tah ótimo.  Positivo   
6     Eu procurei paleta de cabrito mss a Swift nao ...  Detrator   
8             Sempre sou muito bem atendida nesta loja.  Positivo   
...                                                 ...       ...   
1075  A loja de Louveira tem um dos melhores atendim...  Positivo   
1076  O atendimento da moça que trabalha lá foi muit...  Positivo   
1078                                       Mais ofertas    Neutro   
1079  Mantendo a mesma  atenção com os Clientes!!!\n...  Detrator   
1082                 Tudo perfeito  uma ótima  gerente.  Positivo   

                   Categoria  
0     Variedade dos Produtos  
3            Nada a Declarar  
4            Nada a Declarar  
6                    Estoque  
8     Variedade dos Produtos  
...                      ...  
1075  Variedade dos Produtos  
1076                 Estoque  
1078                  Acesso  
1079  Variedade dos Produtos  
1082         Nada a Declarar  

[570 rows x 8 columns]